### **GNC application - Satellite Tracker using Python and AI**
### Project Steps:
1. **Fetch Real-Time TLE Data**: Retrieve TLE data for satellites from CelesTrak.
2. **Calculate Satellite Position**: Use Skyfield to compute the satellite’s location at different times.
3. **Train AI Model**: Develop an LSTM model to predict future satellite positions.
4. **Visualize Satellite Path**: Plot the satellite’s trajectory and predicted positions on an interactive map.
### Required Tools & Libraries:
- **Programming Language**: Python
- **Libraries**:
    - `requests`: Fetching TLE data
    - `skyfield`: Satellite orbit calculations
    - `numpy`, `pandas`: Data processing
    - `tensorflow`/`keras`: AI model (LSTM)
    - `matplotlib`, `plotly`: Visualization
    - `folium`: Interactive map

### Steps to Implement

1. **Fetch Satellite Data from CelesTrak**: Retrieve the TLE data for a satellite, such as the International Space Station (ISS).

In [ ]:
import requests

# Fetch TLE data for ISS
url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=25544&FORMAT=tle"
response = requests.get(url)
tle_data = response.text.strip().split("\n")

# Extract lines
satellite_name = tle_data[0]
tle_line1 = tle_data[1]
tle_line2 = tle_data[2]

print("Satellite:", satellite_name)
print("TLE Line 1:", tle_line1)
print("TLE Line 2:", tle_line2)


### 2- Calculate the Current Orbit using Skyfield
Utilize Skyfield to determine the satellite's position at various times.

In [ ]:
%pip install skyfield
from skyfield.api import load, EarthSatellite

# Load ephemeris data

ts = load.timescale()
satellite = EarthSatellite(tle_line1, tle_line2, satellite_name, ts)

# Get positions for the next 24 hours
times = [ts.utc(2025, 2, 16, h) for h in range(24)]
positions = [satellite.at(t).subpoint() for t in times]

# Extract latitude, longitude, and altitude
latitudes = [p.latitude.degrees for p in positions]
longitudes = [p.longitude.degrees for p in positions]
altitudes = [p.elevation.km for p in positions]


### 3- Train an AI Model for Future Predictions
We employ an LSTM model to analyze the time-series data of satellite positions and forecast future locations.

In [ ]:
%pip install tensorflow
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Convert data into a training set
data = np.array(list(zip(latitudes, longitudes)))
X_train, y_train = data[:-1], data[1:]

# Build the LSTM model
model = keras.Sequential([
    keras.layers.LSTM(50, return_sequences=True, input_shape=(1, 2)),
    keras.layers.LSTM(50),
    keras.layers.Dense(2)  # Output: (latitude, longitude)
])

model.compile(optimizer='adam', loss='mse')

# Reshape data for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
y_train = y_train.reshape((y_train.shape[0], 2))

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=4)


### 4- Predict Future Positions
Using our trained LSTM model, we forecast the satellite's position for the next 6 hours.


In [ ]:
# Predict next position
future_prediction = model.predict(X_train[-1].reshape(1, 1, 2))
predicted_lat, predicted_lon = future_prediction[0]

print(f" Predicted Position in 6 Hours:")
print(f"Latitude: {predicted_lat:.2f}, Longitude: {predicted_lon:.2f}")


### 5- Visualize the Results on a Map
We utilize Folium to plot the satellite's trajectory and its predicted future position.

In [ ]:
%pip install folium
import folium
import webbrowser

# Create a map centered at the first recorded position
m = folium.Map(location=[latitudes[0], longitudes[0]], zoom_start=3)

# Plot actual trajectory
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color="blue").add_to(m)

# Plot predicted position
folium.Marker([predicted_lat, predicted_lon], popup="Predicted Position", icon=folium.Icon(color="red")).add_to(m)

# Save and display the map
m.save("satellite_prediction.html")
print(" Map saved! Open satellite_prediction.html to view the results.")
# Open the saved map in the default web browser
webbrowser.open("satellite_prediction.html")